In [ ]:
#runModel: executes the model that's been built

In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats
import argparse

from modules.RunSPARCED import RunSPARCED

In [4]:
# SBML model we want to import
sbml_file = 'SPARCEDv6.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

In [ ]:
flagD = 1
ts = 30
th = 2
Vn = 1.75E-12
Vc = 5.25E-12

In [ ]:
# for stochastic runs
flagD = 0
flagWr = 1
nmxlsfile = 'GrowthStim_stoc_'

sys.path.insert(0, os.path.abspath(model_output_dir))

species_sheet = np.array([np.array(line.strip().split("\t")) for line in open('Species.txt', encoding='latin-1')])

species_initializations = []
for row in species_sheet[1:]:
    species_initializations.append(float(row[2]))
species_initializations = np.array(species_initializations)

model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

xoutS_all, xoutG_all, tout_all = RunSPARCED(flagD,th,species_initializations,[],Vn,Vc,model)

In [ ]:
# for deterministic runs
flagD = 1
flagWr = 1
nmxlsfile = 'GrowthStim_det_'

sys.path.insert(0, os.path.abspath(model_output_dir))
species_sheet = np.array([np.array(line.strip().split("\t")) for line in open('Species.txt', encoding='latin-1')])

species_initializations = []
for row in species_sheet[1:]:
    species_initializations.append(float(row[2]))

species_initializations = np.array(species_initializations)
species_initializations[np.argwhere(species_initializations <= 1e-6)] = 0.0

model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver()          # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

xoutS_all, xoutG_all, tout_all = RunSPARCED(flagD,th,species_initializations,[],Vn,Vc,model)

In [ ]:
# saves output
columnsS=[ele for ele in model.getStateIds()]
columnsG = columnsS[773:914]
resa = [sub.replace('m_', 'ag_') for sub in columnsG]
resi = [sub.replace('m_', 'ig_') for sub in columnsG]
columnsG2 = np.concatenate((resa, resi), axis=None)
condsSDF = pd.DataFrame(data=xoutS_all,columns=columnsS)
condsSDF.to_excel(nmxlsfile+'S_0.xlsx')
condsSDF = None
condsGDF = pd.DataFrame(data=xoutG_all,columns=columnsG2)
condsGDF.to_excel(nmxlsfile+'G_0.xlsx')
condsGDF = None